<a href="https://colab.research.google.com/github/anasmghouri/Coursera_Capstone/blob/main/The_Battle_of_Neighborhoods_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import re

!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [53]:
url = "https://de.wikipedia.org/wiki/Stadtbezirke_M%C3%BCnchens"
html = requests.get(url).content
df_list = pd.read_html(html)
Mnc_popp = df_list[0]
Mnc_popp.rename(columns={'Nr.': 'No.', 'Stadtbezirk': 'Borough', 'Fläche(km²)': 'Area(km²)', 'Einwohner': 'Population', 'Dichte(Einw./km²)': 'Density(Pop/km²)', 'Ausländer(%)': 'Foreigners(%)'}, inplace=True)
Mnc_popp['Area(km²)'] = Mnc_popp['Area(km²)'].div(100).round(2)
Mnc_popp['Foreigners(%)'] = Mnc_popp['Foreigners(%)'].div(10).round(2)
Mnc_popp.head()

,No.,Borough,Area(km²),Population,Density(Pop/km²),Foreigners(%)
0,1.0,Altstadt-Lehel,3.15,21.126,6.716,26.0
1,2.0,Ludwigsvorstadt-Isarvorstadt,4.40,51.933,11.799,28.3
2,3.0,Maxvorstadt,4.30,51.834,12.060,25.6
3,4.0,Schwabing-West,4.36,68.935,15.800,22.8
4,5.0,Au-Haidhausen,4.22,61.654,14.611,23.5


In [54]:
Mnc_boroughs = df_list[1]
Mnc_boroughs.rename(columns={'Stadt-bezirks-nr.': 'No.', 'Stadtbezirk': 'Borough', 'Stadtbezirksteile (Nr.)': 'Districts'}, inplace=True)
Mnc_boroughs.head()

,No.,Borough,Districts
0,1,Altstadt-Lehel,"Graggenau (1), Angerviertel (2), Hackenviertel..."
1,2,Ludwigsvorstadt-Isarvorstadt,"Gärtnerplatz (1), Deutsches Museum (2), Glocke..."
2,3,Maxvorstadt,"Königsplatz (1), Augustenstraße (2), St. Benno..."
3,4,Schwabing-West,"Neuschwabing (1), Am Luitpoldpark (2), Schwere..."
4,5,Au-Haidhausen,"Maximilianeum (1), Steinhausen (2), Haidhausen..."


In [55]:
# postal code data from: https://www.suche-postleitzahl.org/plz-gebiet/80
Mnc_postal_codes = pd.read_excel(r"https://github.com/anasmghouri/Coursera_Capstone/blob/main/MNC.xlsx?raw=true", sheet_name='PLZ')
Mnc_postal_codes.rename(columns={'PLZ': 'Postal Code', 'Stadtteil': 'Borough'}, inplace=True)
Mnc_postal_codes.head()

,Postal Code,Borough
0,80331,Altstadt-Lehel
1,80331,Ludwigsvorstadt-Isarvorstadt
2,80333,Altstadt-Lehel
3,80333,Maxvorstadt
4,80335,Altstadt-Lehel


In [56]:
# prices data 2020 from: https://wirkaufen24.de/immobilienpreise-munchen/
Mnc_prices = pd.read_excel(r"https://github.com/anasmghouri/Coursera_Capstone/blob/main/MNC.xlsx?raw=true", sheet_name='Price')
del Mnc_prices['Stadt']
Mnc_prices.rename(columns={'Postleitzahl': 'Postal Code', 'Straße': 'Street', 'm² im Ø': 'Objectm²', 'Ø - Preis': 'Object€', 'm² Ø in €': '€/m²'}, inplace=True)
Mnc_prices.head()

,Postal Code,Street,Objectm²,Object€,€/m²
0,80333,Augustenstraße,170.0,2195000,12911.0
1,80333,Barer Straße,66.0,1338000,20272.0
2,80333,Karlstr.,94.0,2600000,27659.0
3,80333,Maximiliansplatz,166.0,22000000,132530.0
4,80333,Richard-Wagner-Str.,211.0,4400000,20853.0
